In [1]:
import pandas as pd
import time

from fasta_processing import dict_align_to_fasta, read_single_fasta

from parse_psi_blast_results import parse_psiblast_xml, filter_psiblast_hits

In [2]:
name_of_blast_res = "../Blast_res/schistosoma_haematobium_psi_blast.xml" # XML2 !!!
df = parse_psiblast_xml(name_of_blast_res)

In [3]:
df_filtered = filter_psiblast_hits(df, min_qc=0.8, min_ident=0.8, min_sbjct_len=500)